In [1]:
import numpy as np
import os
import pandas as pd
from sklearn.manifold import Isomap, LocallyLinearEmbedding, SpectralEmbedding, TSNE
from umap import UMAP
from tqdm import tqdm_notebook
from collections import defaultdict
from sklearn.neighbors import KNeighborsRegressor, NearestNeighbors
from sklearn.metrics import make_scorer, r2_score
from sklearn.multioutput import MultiOutputRegressor, RegressorChain, RegressorMixin
from sklearn.model_selection import cross_val_score, cross_validate, ParameterGrid, GridSearchCV
from sklearn.datasets import make_swiss_roll
from lightgbm import LGBMRegressor
import multiprocessing
from joblib import Parallel, delayed
from utils import mae_score
import matplotlib.pyplot as plt
from utils import project, NPR, filter_paths, unpack_data 
from coranking import coranking_matrix
from coranking.metrics import continuity, LCMC, trustworthiness
from scipy.spatial.distance import jensenshannon, jaccard, braycurtis
from IPython.core.debugger import set_trace
from IPython.display import clear_output


Bad key "text.kerning_factor" on line 4 in
/media/hpc2_storage/ibulygin/miniconda3/envs/fresh/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
X_swiss,y_swiss = make_swiss_roll(n_samples=1000)

# Data PCA

In [3]:
root_separate = './separate_datasets_proj/'
DATASETS = ['AGP_proj_o', 'AGP_proj_f', 'AGP_proj_g', \
            'HMP_proj_o', 'HMP_proj_f', 'HMP_proj_g'] 

paths_separate = filter_paths([os.path.join(root_separate,path) for path in os.listdir(root_separate)], keywords=DATASETS)
int_dims = np.load('./intrinsic_dims_separate_strict.npy', allow_pickle=True).item()

intrinsic_dims = {}
data_pca = {}
for path in paths_separate:
    label = path.split('/')[-1].split('.')[0]
    data_pca[label] = np.genfromtxt(path, delimiter=';')
    intrinsic_dims[label] = int_dims[path]

In [4]:
def cross_validate(X,Z):
    knn = KNeighborsRegressor()
    mo = MultiOutputRegressor(knn)
    mae_scorer = make_scorer(mae_score, greater_is_better=False)
    return -cross_val_score(mo, Z, X, scoring=mae_scorer, cv=3, n_jobs=-1).mean()


def calculate_Q_mae(X, Z):
    
    Q = coranking_matrix(X, Z)
    
    m = X.shape[0]
    UL_cumulative = 0 
    Q_k = []
    LCMC_k = []
    for k in range(0, Q.shape[0]):
        r = Q[k:k+1,:k+1].sum()
        c = Q[:k,k:k+1].sum()
        UL_cumulative += (r+c)
        Qnk = UL_cumulative/((k+1)*m) # (k+1)
        Q_k.append(Qnk)
        LCMC_k.append(Qnk - ((k+1)/(m-1)))
    
    argmax_k = np.argmax(LCMC_k)
    k_max = np.arange(1.,m)[argmax_k]
    Q_loc = (1./k_max)*np.sum(Q_k[:argmax_k+1])
    Q_glob = (1./(m-k_max))*np.sum(Q_k[argmax_k+1:])
    
    mae = cross_validate(X, Z)
    
    return Q_loc, Q_glob, mae
    

In [5]:
def transform(method, X, dim, parameters, scorer):
#     try:
    model_inst = method(n_components=dim, **parameters, n_jobs=-1)
    Z = model_inst.fit_transform(X)
    score = scorer(X,Z)
#     except:
#         score = [None, None, None]
    return score

In [6]:
# %%time
# # the problem is with AGP_proj_o, with others seems OK
# model_param_grid = {'affinity':['nearest_neighbors', 'rbf'],
#                      'n_neighbors':[3,5,10,15,25],
#                       'gamma':[1e-4,1e-3,None,1e-1],
#                       'random_state':[42]
#                       }
# isomap_results = Parallel(n_jobs=-1)(delayed(transform)(SpectralEmbedding,
#                                                         X=data_pca['AGP_proj_f'], # AGP_proj_o
#                                                         dim=5,
#                                                         parameters=model_params,
#                                                         scorer=calculate_Q_mae) \
#                                                         for model_params in tqdm_notebook(list(ParameterGrid(model_param_grid)))) 

In [7]:
# %%time
# # the problem is with AGP_proj_o, with others seems OK
# model_param_grid = {'affinity':['nearest_neighbors', 'rbf'],
#                      'n_neighbors':[3,5,10,15,25],
#                       'gamma':[1e-4,1e-3,None,1e-1],
#                       'random_state':[42]
#                       }
# isomap_results =[]
# for model_params in tqdm_notebook(list(ParameterGrid(model_param_grid))):
#     print(model_params)
#     print('\n')
#     isomap_results.append(transform(SpectralEmbedding,
#                             X=data_pca['AGP_proj_o'], # AGP_proj_o
#                             dim=5,
#                             parameters=model_params,
#                             scorer=calculate_Q_mae))


In [8]:
# spectral_results_dict = {}
# spectral_results_dict['params'] = model_param_grid
# spectral_results_dict['results'] = spectral_results

In [9]:
# np.save('./manifold_learning_results_separate/AGP_proj_o_spectral_results', spectral_results_dict)

In [10]:
# some_params = [2,3,5,10] #['standard', 'hessian', 'modified', 'ltsa']
# fig, axes = plt.subplots(ncols=len(some_params), nrows=1, figsize=(len(some_params)*5,5), dpi=300)
# for i,prm in enumerate(some_params):
#     se = LocallyLinearEmbedding(n_components=2, n_neighbors=prm, method='ltsa', eigen_solver='dense') # modified
#     Z = se.fit_transform(X_swiss)
#     Q_loc, Q_glob, mae =calculate_Q_mae(X_swiss, Z) 
#     axes[i].scatter(Z[:,0], Z[:,1],c=y_swiss)
#     axes[i].set_title(f'Q_loc: {Q_loc}, \n Q_glob: {Q_glob}, \n MAE: {mae}')   
# plt.tight_layout()
# plt.show() 

# # 2 * (2 + 3) / 2

In [11]:
Isomap(), SpectralEmbedding(), LocallyLinearEmbedding(), UMAP(), TSNE()

(Isomap(),
 SpectralEmbedding(),
 LocallyLinearEmbedding(),
 UMAP(dens_frac=0.0, dens_lambda=0.0),
 TSNE())

In [ ]:
mf_models = {#'isomap':{'class':Isomap, 'params_grid':{'n_neighbors':[3,5,10,15,25],
#                                                       'p':[1,2],
#                                                       'metric':['minkowski']
#                                                       }
#                       },
            'spectral':{'class':SpectralEmbedding,'params_grid':{'affinity':['nearest_neighbors', 'rbf'],
                                                                 'n_neighbors':[3,5,10,15,25],
                                                                  'gamma':[1e-4,1e-3,None,1e-1],
                                                                  'random_state':[42],
                                                                  }
                        },
#             'lle':{'class':LocallyLinearEmbedding, 'params_grid':{'method':['ltsa','modified'],
#                                                                    'n_neighbors':[10,15,25,50],
#                                                                    'random_state':[42],
#                                                                    'eigen_solver':['dense']}
#                   },
#             'umap':{'class':UMAP, 'params_grid':{'n_neighbors':[3,5,10,15,25],
#                                                  'min_dist':[0.1, 0.2, 0.4, 0.6, 0.8],
#                                                  'random_state':[42],
#                                                  'metric':['euclidean','manhattan']}
#                    },
#             'tsne':{'class':TSNE, 'params_grid':{'perplexity':[3,5,10,15,25],
#                                                  'random_state':[42],
#                                                  'early_exaggeration':[1,10,20,50],
#                                                  'init':['pca'],
#                                                  'metric':['euclidean','manhattan']
#                                                  }
#                    } 
            }

results = defaultdict(dict)

for label, X in tqdm_notebook(data_pca.items()):

        dim = intrinsic_dims[label][-1]
        
        for mf_type, model in mf_models.items():
            print(label, dim, mf_type)
            if mf_type=='lle':
                model['params_grid']['n_neighbors'] = list(filter(lambda x: x>dim+1, model['params_grid']['n_neighbors']))
            
            model_class = model['class']
            model_param_grid = model['params_grid']
            results[label][mf_type] = {}
            
#             scores = Parallel(n_jobs=10)(delayed(transform)(model_class,
#                                                             X, 
#                                                             dim,
#                                                             model_params,
#                                                             calculate_Q_mae) \
#                                                             for model_params in list(ParameterGrid(model_param_grid))) 
            scores = []
            for model_params in list(ParameterGrid(model_param_grid)):
                print(model_params)
                scores.append(transform(model_class,
                                        X, 
                                        dim,
                                        model_params,
                                        calculate_Q_mae))
    

            results[label][mf_type]['scores'] = scores
            results[label][mf_type]['params'] = model_param_grid
            clear_output()
                
np.save('mf_learning_metrics_results', results)

AGP_proj_o 5 spectral
{'affinity': 'nearest_neighbors', 'gamma': 0.0001, 'n_neighbors': 3, 'random_state': 42}
{'affinity': 'nearest_neighbors', 'gamma': 0.0001, 'n_neighbors': 5, 'random_state': 42}
{'affinity': 'nearest_neighbors', 'gamma': 0.0001, 'n_neighbors': 10, 'random_state': 42}
{'affinity': 'nearest_neighbors', 'gamma': 0.0001, 'n_neighbors': 15, 'random_state': 42}
{'affinity': 'nearest_neighbors', 'gamma': 0.0001, 'n_neighbors': 25, 'random_state': 42}
{'affinity': 'nearest_neighbors', 'gamma': 0.001, 'n_neighbors': 3, 'random_state': 42}
{'affinity': 'nearest_neighbors', 'gamma': 0.001, 'n_neighbors': 5, 'random_state': 42}
{'affinity': 'nearest_neighbors', 'gamma': 0.001, 'n_neighbors': 10, 'random_state': 42}
{'affinity': 'nearest_neighbors', 'gamma': 0.001, 'n_neighbors': 15, 'random_state': 42}
{'affinity': 'nearest_neighbors', 'gamma': 0.001, 'n_neighbors': 25, 'random_state': 42}
{'affinity': 'nearest_neighbors', 'gamma': None, 'n_neighbors': 3, 'random_state': 42}


In [ ]:
# scores = np.array(results[label][mf_type]['scores'])
# params = list(ParameterGrid(results[label][mf_type]['params']))

# plt.figure(figsize=(15,15), dpi=200)
# c = plt.scatter(scores[:,0], scores[:,1], c=scores[:,-1])
# for txt, xy in zip(np.arange(len(scores)), scores[:,:2]):
#     plt.annotate(txt, xy)
# plt.xlabel('Q_loc')
# plt.ylabel('Q_glob')
# plt.colorbar()
# plt.show()

Isomap: {'metric': 'minkowski', 'n_neighbors': 3, 'p': 1}, AGP_o

In [19]:
best_index = 6
best_params = params[best_index]
best_params, scores[best_index]

({'method': 'modified', 'n_neighbors': 12, 'random_state': 42},
 array([0.60696179, 0.86831541, 0.63519375]))

In [20]:
lle = LocallyLinearEmbedding(n_components=dim, **best_params)
Z = lle.fit_transform(X)

In [33]:
knn = LGBMRegressor()
# mo = MultiOutputRegressor(knn)
mo = RegressorChain(knn)

prefix = 'base_estimator___'
params={'boosting_type':['gbdt', 'dart', 'goss', 'rf'],
       'learning_rate':[0.1, 0.001, 0.0001],
       'reg_alpha':[0.001, 0.1, 1., 10.],
       'reg_lambda':[0.001, 0.1, 1., 10],
       'random_state':[42]}

params_ = {}
for k,v in params.items():
    params_[prefix+k] = v

gs = GridSearchCV(mo, params_, cv=3, refit=False, n_jobs=-1, scoring=mae_scorer)
gs.fit(Z,X)

# mae_scorer = make_scorer(mae_score, greater_is_better=False)
# -cross_val_score(mo, Z, X, scoring=mae_scorer, cv=3, n_jobs=-1).mean()

GridSearchCV(cv=3, estimator=RegressorChain(base_estimator=LGBMRegressor()),
             n_jobs=-1,
             param_grid={'base_estimator___boosting_type': ['gbdt', 'dart',
                                                            'goss', 'rf'],
                         'base_estimator___learning_rate': [0.1, 0.001, 0.0001],
                         'base_estimator___random_state': [42],
                         'base_estimator___reg_alpha': [0.001, 0.1, 1.0, 10.0],
                         'base_estimator___reg_lambda': [0.001, 0.1, 1.0, 10]},
             refit=False,
             scoring=make_scorer(mae_score, greater_is_better=False))

### Visualization

In [ ]:
# for label, mf_dict in results.items():
    
#     m = len(mf_dict)
#     fig, axes = plt.subplots(ncols=m, nrows=1, figsize=(m*5,5), dpi=200)
#     for i,(mf_type, metrics) in enumerate(mf_dict.items()):
        
#         if len(metrics) == 0:
#             continue
        
#         scores = np.array(metrics['scores'])
#         model_param_grid = metrics['params']
#         dim = intrinsic_dims[label][-1]
#         mae = scores[:,-1]
#         Q_ = scores[:,:2]
        
#         sc = axes[i].scatter(Q_[:,0], Q_[:,1], c=mae)
#         axes[i].set_xlabel('Q_loc')
#         axes[i].set_ylabel('Q_glob')
#     fig.colorbar(sc, orientation='vertical')
#     fig.suptitle(label, fontsize=16, color='blue')
    
# # plt.colorbar()
# plt.tight_layout()
# plt.subplots_adjust(top=0.85)
# plt.show()

### Saving

In [ ]:
# SAVE = True

# for label,label_dict in results.items(): 
#     for method_name, method_dict in label_dict.items(): 
#         recommended_intrinsic_dims = intrinsic_dims[label2path[label]]
        
#         d = np.genfromtxt(path, delimiter=';')
#         dims = intrinsic_dims[path]
#         d1 = min(2, min(dims))
#         d2 = min(d.shape[1], max(dims)+6)
#         DIM_RANGE = np.arange(d1, d2)

#         mae = np.array(method_dict['knn_neg_mae'])

#         ind = np.intersect1d(DIM_RANGE, recommended_intrinsic_dims, assume_unique=True, return_indices=True)[1].tolist()
#         i = ind[np.array(mae)[ind].argmin()]

#         d_trans = method_dict['dataset_transformed'][i]
        
#         if SAVE:
#             if MERGED:
#                 name = f"./merged_datasets_transformed/{label}_{method_name}.csv"
#             else:
#                 name = f"./separate_datasets_transformed/{label}_{method_name}.csv"
#             np.savetxt(name, d_trans, delimiter=";")

In [ ]:
# for label,label_dict in results.items(): 
#     for method_name, method_dict in label_dict.items(): 

#         d_trans = method_dict['dataset_transformed'][i]
# #         print(label, method_name, d_trans.shape)
#         name = f"./separate_datasets_transformed/{label}_{method_name}.csv"
#         np.savetxt(name, d_trans, delimiter=";")

# Original with precomputed distances

In [ ]:
data_orig = {}
distances_orig = defaultdict(dict)

for dataset in ['AGP', 'HMP']:
    for tax_level in ['o', 'f', 'g']:
        dataframe_out = pd.read_csv(f'{dataset}/pivot_{tax_level}_normalized.csv', skipinitialspace=True, sep=';', engine='python')
        dataframe_out = dataframe_out.drop('Unnamed: 0', axis = 1).values
        
        name = '_'.join([dataset, tax_level])
        data_orig[name] = dataframe_out
        distances_orig['JS'][name] = np.load(f'./distances/orig_JS_{dataset}_{tax_level}.npy')
        distances_orig['BC'][name] = np.load(f'./distances/orig_BC_{dataset}_{tax_level}.npy')        

In [ ]:
# mf_models = {'isomap':{'class':Isomap, 'params_grid':{'n_neighbors':[3,5,10,15,25],
#                                                       'p':[1,2],
#                                                       'metric':['minkowski']
#                                                       }
#                       },
#             'spectral':{'class':SpectralEmbedding,'params_grid':{'affinity':['nearest_neighbors', 'rbf'],
#                                                                  'n_neighbors':[3,5,10,15,25],
#                                                                   'gamma':[1e-4,1e-3,None,1e-1],
#                                                                   'random_state':[42]
#                                                                   }
#                         },
#             'lle':{'class':LocallyLinearEmbedding, 'params_grid':{'method':['ltsa','modified'],
#                                                                    'n_neighbors':[10,15,25,50],
#                                                                    'random_state':[42],
#                                                                    'eigen_solver':['dense']}
#                   },
#             'umap':{'class':UMAP, 'params_grid':{'n_neighbors':[3,5,10,15,25],
#                                                  'min_dist':[0.1, 0.2, 0.4, 0.6, 0.8],
#                                                  'random_state':[42],
#                                                  'metric':['euclidean','manhattan']}
#                    },
#             'tsne':{'class':TSNE, 'params_grid':{'perplexity':[3,5,10,15,25],
#                                                  'random_state':[42],
#                                                  'early_exaggeration':[1,10,20,50],
#                                                  'init':['pca'],
#                                                  'metric':['euclidean','manhattan']
#                                                  }
#                    } 
#             }

# results = defaultdict(dict)

# for label, X in tqdm_notebook(data_orig.items()):
# # for label, X in tqdm_notebook([('AGP_proj_f', data_pca['AGP_proj_f'])]):

#         dim = intrinsic_dims[label][-1]
        
#         for distance_name, S in distances_orig.items():
        
#         for mf_type, model in mf_models.items():
#             print(label, dim, mf_type)
#             if mf_type=='lle':
#                 model['params_grid']['n_neighbors'] = list(filter(lambda x: x>dim+1, model['params_grid']['n_neighbors']))
            
#             model_class = model['class']
#             model_param_grid = model['params_grid']
#             results[label][mf_type] = {}
            
#             scores = Parallel(n_jobs=10)(delayed(transform)(model_class,
#                                                             X, 
#                                                             dim,
#                                                             model_params,
#                                                             calculate_Q_mae) \
#                                                             for model_params in list(ParameterGrid(model_param_grid))) 

#             results[label][mf_type]['scores'] = scores
#             results[label][mf_type]['params'] = model_param_grid
#             clear_output()
                
# np.save('mf_learning_metrics_results', results)